In [1]:
import re

def check_password_strength(password: str) -> bool:
    """
    Checks password strength and prints specific feedback for unmet criteria.
    Returns True if password is strong, otherwise False.
    """
    is_strong = True

    if len(password) < 8:
        print("❌ Password should be at least 8 characters long.")
        is_strong = False

    if not re.search(r'[A-Z]', password):
        print("❌ Password should contain at least one uppercase letter (A-Z).")
        is_strong = False

    if not re.search(r'[a-z]', password):
        print("❌ Password should contain at least one lowercase letter (a-z).")
        is_strong = False

    if not re.search(r'\d', password):
        print("❌ Password should contain at least one digit (0-9).")
        is_strong = False

    if not re.search(r'[!@#$%^&*(),.?":{}|<>_\[\]\/\\\-+=~`]', password):
        print("❌ Password should contain at least one special character (e.g., !, @, #, etc.).")
        is_strong = False

    return is_strong

# Main script
if __name__ == "__main__":
    password = input("Enter your password to check strength: ")

    print("\nChecking password strength...\n")
    if check_password_strength(password):
        print("✅ Your password is strong.")
    else:
        print("\n🔒 Please fix the issues above to strengthen your password.")


Enter your password to check strength: Astha65aa

Checking password strength...

❌ Password should contain at least one special character (e.g., !, @, #, etc.).

🔒 Please fix the issues above to strengthen your password.


In [4]:


import psutil
import time
import logging
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Thresholds
CPU_THRESHOLD = 80
MEMORY_THRESHOLD = 80
DISK_THRESHOLD = 90
CHECK_INTERVAL = 5  # in seconds

# Email Config
EMAIL_SENDER = "your_email@example.com"
EMAIL_RECEIVER = "admin@example.com"
SMTP_SERVER = "smtp.example.com"
SMTP_PORT = 587
SMTP_USER = "your_email@example.com"
SMTP_PASSWORD = "your_email_password"

# Logging
logging.basicConfig(
    filename="system_monitor.log",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECEIVER
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SMTP_USER, SMTP_PASSWORD)
        server.send_message(msg)
        server.quit()
    except Exception as e:
        logging.error(f"Failed to send alert email: {e}")

def monitor_system():
    print("🔍 Starting system monitoring... Press Ctrl+C to stop.")
    try:
        while True:
            try:
                cpu = psutil.cpu_percent(interval=1)
                memory = psutil.virtual_memory().percent
                disk = psutil.disk_usage('/').percent

                print(f"CPU: {cpu}% | Memory: {memory}% | Disk: {disk}%")

                alerts = []

                if cpu > CPU_THRESHOLD:
                    alert = f"⚠️ CPU usage is high: {cpu}%"
                    print(alert)
                    alerts.append(alert)

                if memory > MEMORY_THRESHOLD:
                    alert = f"⚠️ Memory usage is high: {memory}%"
                    print(alert)
                    alerts.append(alert)

                if disk > DISK_THRESHOLD:
                    alert = f"⚠️ Disk usage is high: {disk}%"
                    print(alert)
                    alerts.append(alert)

                if alerts:
                    email_subject = "🚨 System Resource Alert"
                    email_body = "\n".join(alerts)
                    send_email(email_subject, email_body)

                time.sleep(CHECK_INTERVAL)

            except Exception as e:
                logging.error(f"Monitoring error: {e}")
                print("An error occurred. Check logs.")

    except KeyboardInterrupt:
        print("\n🛑 Monitoring stopped by user. Exiting.")

if __name__ == "__main__":
    monitor_system()


🔍 Starting system monitoring... Press Ctrl+C to stop.
CPU: 9.7% | Memory: 84.9% | Disk: 30.5%
⚠️ Memory usage is high: 84.9%
CPU: 4.8% | Memory: 84.9% | Disk: 30.5%
⚠️ Memory usage is high: 84.9%
CPU: 6.0% | Memory: 84.9% | Disk: 30.5%
⚠️ Memory usage is high: 84.9%
CPU: 3.7% | Memory: 85.0% | Disk: 30.5%
⚠️ Memory usage is high: 85.0%
CPU: 5.5% | Memory: 85.0% | Disk: 30.5%
⚠️ Memory usage is high: 85.0%
CPU: 1.5% | Memory: 85.0% | Disk: 30.5%
⚠️ Memory usage is high: 85.0%
CPU: 10.8% | Memory: 85.5% | Disk: 30.5%
⚠️ Memory usage is high: 85.5%
CPU: 2.1% | Memory: 86.4% | Disk: 30.5%
⚠️ Memory usage is high: 86.4%
CPU: 19.3% | Memory: 86.7% | Disk: 30.5%
⚠️ Memory usage is high: 86.7%
CPU: 10.6% | Memory: 86.9% | Disk: 30.5%
⚠️ Memory usage is high: 86.9%
CPU: 12.0% | Memory: 86.8% | Disk: 30.5%
⚠️ Memory usage is high: 86.8%
CPU: 7.0% | Memory: 86.5% | Disk: 30.5%
⚠️ Memory usage is high: 86.5%
CPU: 6.2% | Memory: 90.1% | Disk: 30.5%
⚠️ Memory usage is high: 90.1%
CPU: 15.2% | Memory

In [ ]:
import os
import yaml
import json
import sqlite3
from flask import Flask, jsonify

CONFIG_FILE = 'docker-compose.yml'
DB_FILE = 'config_data.db'
TABLE_NAME = 'config_json'

def read_config_file(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Configuration file '{path}' not found.")
    
    try:
        with open(path, 'r') as f:
            return yaml.safe_load(f)
    except Exception as e:
        raise IOError(f"Failed to read the file: {e}")

def extract_services(config):
    services_info = []
    services = config.get('services', {})
    for name, details in services.items():
        service = {
            'name': name,
            'image': details.get('image', 'N/A'),
            'build': details.get('build', 'N/A'),
            'ports': details.get('ports', [])
        }
        services_info.append(service)
    return services_info

def store_json_in_db(data):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            data TEXT NOT NULL
        )
    ''')
    cursor.execute(f'INSERT INTO {TABLE_NAME} (data) VALUES (?)', (json.dumps(data),))
    conn.commit()
    conn.close()

def fetch_data_from_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(f'SELECT data FROM {TABLE_NAME} ORDER BY id DESC LIMIT 1')
    result = cursor.fetchone()
    conn.close()
    return json.loads(result[0]) if result else {}


In [ ]:
try:
    config_data = read_config_file(CONFIG_FILE)
    extracted_data = extract_services(config_data)
    store_json_in_db(extracted_data)
    print("✅ Configuration data successfully extracted and stored.")
except Exception as e:
    print(f"❌ Error: {e}")


In [ ]:
# Simulate GET request to `/config`
data = fetch_data_from_db()
import pprint
pprint.pprint(data)


In [ ]:
app = Flask(__name__)

@app.route('/config', methods=['GET'])
def get_config():
    data = fetch_data_from_db()
    return jsonify(data)

# Run Flask app (non-blocking for notebook - use use_reloader=False)
app.run(debug=False, port=5001, use_reloader=False)
